##### multiquery +rag Fusion

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("openai api 키가 없습니다. 한번더 확인 부탁드립니다.")

# 환경변수 등록
os.environ['OPENAI_API_KEY']=openai_api_key

In [ ]:
import bs4
from langchain_classic import hub
from langchain_classic.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

#### INDEXING ####

loader = WebBaseLoader(
    web_paths=("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    ),
)
docs = loader.load()


# Split: chunk_size=300, chunk_overlap=50
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

# Make splits, split_documents(docs)
splits = text_splitter.split_documents(docs)

# Index: from_documents(documents=splits, embedding=OpenAIEmbeddings())
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 검색기: as_retriever()
retriever = vectorstore.as_retriever()


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [ ]:
# Multiquery 생성:ChatPromptTemplate.from_template(template)

template = """당신은 주어진 하나의 질문을 기반으로 여러 검색 쿼리를 생성하는 유용한 조수입니다. \n
다음 질문과 관련된 여러 검색 쿼리를 생성하세요: {question} \n
출력 (4개의 쿼리):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(model_name="gpt-4o-mini",temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)
generated_query = generate_queries.invoke("집값의 향방?")
generated_query

['1. "2023년 집값 전망"',
 '2. "집값 상승 원인과 예측"',
 '3. "한국 부동산 시장 동향 2023"',
 '4. "집값 하락 가능성 분석"']

In [ ]:
from langchain_classic.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, top_n=2):
    """ 
    여러 개의 순위가 매겨진 문서 리스트를 받아, RRF(Reciprocal Rank Fusion) 공식을 사용하여
    문서의 최종 순위를 계산하는 함수입니다. k는 RRF 공식에서 사용되는 선택적 파라미터이며,
    top_n은 반환할 우선순위가 높은 문서의 개수입니다.
    """
    
    # 각 고유한 문서에 대한 점수를 저장할 딕셔너리를 초기화합니다.
    fused_scores = {}

    # 순위가 매겨진 문서 리스트를 순회합니다.
    for docs in results:
        # 리스트 내의 각 문서와 그 문서의 순위를 가져옵니다.
        for rank, doc in enumerate(docs):
            # 문서를 문자열로 변환
            doc_str = dumps(doc)
            # 해당 문서가 아직 딕셔너리에 없으면 초기 점수 0으로 추가합니다.
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # 문서의 현재 점수를 가져옵니다 (이전에 계산된 점수).
            previous_score = fused_scores[doc_str]
            # RRF 공식을 사용하여 문서의 점수를 업데이트합니다: 1 / (순위 + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # 문서들을 계산된 점수에 따라 내림차순으로 정렬하여 최종적으로 재정렬된 결과를 얻습니다.
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # 재정렬된 결과에서 우선순위가 높은 top_n 개의 문서만 반환합니다.
    return reranked_results[:top_n]




In [5]:
# RAG-Fusion 체인을 구성합니다.
# generate_queries: 질문에 대해 여러 검색 쿼리를 생성합니다.
# retriever.map(): 생성된 쿼리로 관련 문서들을 검색합니다.
# reciprocal_rank_fusion: 검색된 문서들을 RRF 알고리즘을 통해 결합하여 최종 순위를 계산합니다.
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

# 체인을 실행하여 질문에 대한 검색된 문서들을 가져옵니다.
question = "향후 집값에 대해서 알려줘"
docs = retrieval_chain_rag_fusion.invoke({"question": question})

# 검색된 고유 문서들의 개수를 출력합니다.
print(len(docs))

# 검색된 고유 문서들을 출력합니다.
docs

2


C:\Users\samsung\AppData\Local\Temp\ipykernel_24988\1971723636.py:29: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  (loads(doc), score)


[(Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='지난해 서울 아파트값이 19년 만에 가장 크게 오른 것으로 집계됐다. 지난 반년 새 대규모 부동산 정책이 잇따라 나왔지만, 외려 ‘똘똘한 한 채’ 현상 가중으로 서울 집값 상승세는 꺾이지 않았다. ‘미친 집값’이란 \n\n\n중앙일보\n\n1시간전'),
  0.06612903225806452),
 (Document(metadata={'source': 'https://news.naver.com/section/101'}, page_content='뉴시스\n\n1시간전\n\n\n\n\n\n\n\n\n서울 집값, 마지막 주까지 올랐다…연간 상승률 19년 만 최대'),
  0.03278688524590164)]

In [6]:
# RAG
template = """다음 맥락을 바탕으로 질문에 답변하세요:

{context}

질문: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke(question)

"현재 서울 아파트값이 19년 만에 가장 크게 오른 상황이며, 대규모 부동산 정책에도 불구하고 상승세가 꺾이지 않고 있는 것으로 보입니다. 특히 '똘똘한 한 채' 현상이 가중되고 있어, 특정 지역이나 고급 아파트에 대한 수요가 계속해서 높아지고 있습니다. 이러한 추세가 지속된다면 향후 집값은 여전히 상승할 가능성이 높습니다. 다만, 정부의 추가적인 부동산 정책이나 경제 상황에 따라 변동성이 클 수 있으므로 주의가 필요합니다."